# Mouting the Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Importing Necessary Libraries

In [2]:
import numpy as np
import pandas as pd
import os
from os import listdir
import cv2

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
from PIL import Image
from glob import glob
from skimage.io import imread

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

import torch 
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torch.optim as optim

import time
import copy
from tqdm import tqdm_notebook as tqdm

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from lightgbm import LGBMClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

print('Imports complete')

Imports complete


In [3]:
base_dir= '/content/drive/MyDrive/Colab Notebooks'
print(os.listdir(base_dir))

['Untitled0.ipynb', 'Untitled (1)', 'Untitled1.ipynb', 'Untitled2.ipynb', 'Untitled3.ipynb', 'Untitled4.ipynb', 'B21ME029_icsass6.ipynb', 'B21ME029ass8.ipynb', 'Untitled5.ipynb', 'B21ME029Ass7.ipynb', 'Untitled6.ipynb', 'Untitled7.ipynb', 'Copy of IML lab1.ipynb', 'Copy of PRML lab1.ipynb', 'PRML lab1.ipynb', 'Untitled', 'B21CS091_PRML_lab4.ipynb', 'PRML_lab_3.ipynb', 'B21CS091_inlab4.ipynb', 'B21CS091_PRML_lab 2.ipynb', 'B21CS091_PRML_LAB_5.ipynb', 'B21CS091_LAB_Assignment_6.ipynb', 'Copy of B21CS091_prml_inlab7.ipynb', 'B21CS091_PRML7.ipynb', 'Copy of B21CS091_PRML7.ipynb', 'Untitled8.ipynb', 'Untitled9.ipynb', 'Copy of neuralnetwork_classification.ipynb', 'B21CS091_Lab_Assignment_10.ipynb', 'B21CS091_PRML_9.ipynb', 'B21CS091_Lab_Assignment_8.ipynb', 'PRML_MINOR_PROJECT.ipynb', 'YieldStrength_predict.ipynb', 'Untitled10.ipynb', 'Elongation.ipynb', 'Manipulated.ipynb', 'B21CS091_Lab_Assignment_11.ipynb', 'histopathologic-cancer-detection.zip', 'notebook7124a395c6.ipynb', 'PRML-Major-P

In [4]:
!unzip '/content/drive/MyDrive/Colab Notebooks/histopathologic-cancer-detection.zip' > /dev/null

In [5]:
train_path = '/content/train'
test_path = '/content/test'
train_files = os.listdir(train_path)
test_files = os.listdir(test_path)

In [9]:
labels_ = pd.read_csv('/content/train_labels.csv')
labels_.head()

,id,label
0,f38a6374c348f90b587e046aac6079959adf3835,0
1,c18f2d887b7ae4f6742ee445113fa1aef383ed77,1
2,755db6279dae599ebb4d39a9123cce439965282d,0
3,bc3f0c64fb968ff4a8bd33af6971ecae77c75e08,0
4,068aba587a4950175d04c680d38943fd488d6a9d,0


# Sampling Random Statifies Data 

In [26]:
sampled, remainder = train_test_split(labels_, stratify=labels_.label, train_size=0.025)

In [27]:
sampled.to_csv('/content/labels_sampled.csv')

In [29]:
labels = pd.read_csv("/content/labels_sampled.csv")
labels.head()

,Unnamed: 0,id,label
0,175486,90ecebe5f14ba5d63bb4ba24bf6472391c406278,1
1,102041,07638160aac5b60d4109dd31766c228127c83dbe,1
2,194160,0e509fcddf1eae5930080bcaf0fb9822dcfe0c06,0
3,126917,8e83d25c31702bd93fe713d7098bf8d8885cf0e9,0
4,196032,20a1307ef044d93305d6492382aaaa4281bb95d9,1


In [15]:
from PIL import Image
IMG_WIDTH=96
IMG_HEIGHT=96
img_folder='/content/train'

In [16]:
import random
import matplotlib.image as mpimg
%matplotlib inline

In [31]:
def create_dataset_PIL(img_folder,labels):
    
    img_data_array=[]
    class_name=[]
    for i, file in enumerate(labels['id']):
           
            image= np.array(Image.open("/content/train/" + file + ".tif"))
            image = image.astype('float32')
            image /= 255  
            img_data_array.append(image)
            class_name.append(labels['label'][i])
    return img_data_array , class_name
PIL_img_data, class_name=create_dataset_PIL(img_folder,labels)

In [32]:
print(len(PIL_img_data))
print(PIL_img_data[0].shape)

5500
(96, 96, 3)


In [33]:
flatten=[]
for i in PIL_img_data:
  flatten.append(i.flatten())
flatten = np.array(flatten)
flatten.shape


(5500, 27648)

#Principal Component Analysis




In [41]:
pca90 = PCA(n_components = 0.9)
pca90.fit(flatten,class_name)
pca_data90 = pca90.transform(flatten)

In [44]:
lda = LDA(store_covariance= True)
lda.fit(pca_data90,class_name)
lda_data90 = lda.transform(pca_data90)

# **Models without Applying PCA**

## Linear SVC

In [46]:
X_train,X_test,y_train,y_test=train_test_split(flatten,class_name,stratify=class_name,train_size=0.8,random_state=10)

In [47]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [48]:
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
     

from sklearn.pipeline import make_pipeline
     

clf = make_pipeline(StandardScaler(), LinearSVC(random_state=0, tol=1e-5))
clf.fit(X_train,y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearsvc', LinearSVC(random_state=0, tol=1e-05))])

In [49]:
y_pred=clf.predict(X_test)
acc=classification_report(y_test, y_pred)
print(acc)

              precision    recall  f1-score   support

           0       0.67      0.67      0.67       654
           1       0.52      0.51      0.52       446

    accuracy                           0.61      1100
   macro avg       0.59      0.59      0.59      1100
weighted avg       0.61      0.61      0.61      1100



# **KNN**

In [51]:
from sklearn.neighbors import KNeighborsClassifier
     

KNN_ = KNeighborsClassifier(n_neighbors=5)
KNN_.fit(X_train, y_train)
     
KNeighborsClassifier()

y_pred=KNN_.predict(X_test)
acc=classification_report(y_test, y_pred)
print(acc)

              precision    recall  f1-score   support

           0       0.78      0.44      0.56       654
           1       0.50      0.82      0.62       446

    accuracy                           0.59      1100
   macro avg       0.64      0.63      0.59      1100
weighted avg       0.66      0.59      0.58      1100



# **Using RBF SVC**

In [52]:
from sklearn.svm import SVC
     

clf = make_pipeline(StandardScaler(), SVC(random_state=0, gamma='auto'))
clf.fit(X_train,y_train)
     
Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto', random_state=0))])

y_pred=clf.predict(X_test)
acc=classification_report(y_test, y_pred)
print(acc)

              precision    recall  f1-score   support

           0       0.80      0.88      0.84       654
           1       0.79      0.68      0.73       446

    accuracy                           0.80      1100
   macro avg       0.80      0.78      0.78      1100
weighted avg       0.80      0.80      0.79      1100



# **Using LGBM**

In [53]:

lgbm = LGBMClassifier()
lgbm.fit(X_train,y_train)
     
LGBMClassifier()

y_pred = lgbm.predict(X_test)
acc = classification_report(y_test,y_pred)
print(acc)

              precision    recall  f1-score   support

           0       0.79      0.87      0.83       654
           1       0.77      0.66      0.71       446

    accuracy                           0.78      1100
   macro avg       0.78      0.76      0.77      1100
weighted avg       0.78      0.78      0.78      1100



# **Models with PCA(0.9) and LCA**

### **Linear Support Vector Classifier**

In [54]:
X_train,X_test,y_train,y_test=train_test_split(lda_data90,class_name,stratify=class_name,train_size=0.8,random_state=10)

In [55]:
model = Pipeline((('scaler',StandardScaler()),('linear svc', LinearSVC(random_state=0, tol=1e-5))))
model.fit(X_train,y_train)

y_pred=model.predict(X_test)
acc=classification_report(y_test, y_pred)
print(acc)
     

              precision    recall  f1-score   support

           0       0.84      0.87      0.86       654
           1       0.80      0.76      0.78       446

    accuracy                           0.82      1100
   macro avg       0.82      0.81      0.82      1100
weighted avg       0.82      0.82      0.82      1100



**Grid Search**

In [56]:
clf = GridSearchCV(model,{'linear svc__C':[0.25,2.5,25,250]})
clf.fit(X_train,y_train)
clf.best_params_

{'linear svc__C': 250}

In [57]:

clf = GridSearchCV(model,{'linear svc__C':[0.25,2.5,25,250]})
clf.fit(X_train,y_train)
clf.best_params_

{'linear svc__C': 250}

In [58]:
clf.cv_results_

{'mean_fit_time': array([0.09293203, 0.29559917, 0.18399644, 0.15642676]),
 'std_fit_time': array([0.02243043, 0.13841283, 0.03072901, 0.01009699]),
 'mean_score_time': array([0.01251564, 0.00491548, 0.00393882, 0.00235829]),
 'std_score_time': array([2.27828699e-03, 2.74608001e-03, 2.17726608e-03, 5.43944765e-05]),
 'param_linear svc__C': masked_array(data=[0.25, 2.5, 25, 250],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'linear svc__C': 0.25},
  {'linear svc__C': 2.5},
  {'linear svc__C': 25},
  {'linear svc__C': 250}],
 'split0_test_score': array([0.79886364, 0.79886364, 0.8       , 0.79545455]),
 'split1_test_score': array([0.80795455, 0.80795455, 0.80795455, 0.80909091]),
 'split2_test_score': array([0.83295455, 0.83295455, 0.83295455, 0.83522727]),
 'split3_test_score': array([0.83522727, 0.83522727, 0.83522727, 0.83636364]),
 'split4_test_score': array([0.80113636, 0.80113636, 0.80113636, 0.80227273]),
 'mean_

## **KNN**

In [59]:
from sklearn.neighbors import KNeighborsClassifier
     

KNN_ = KNeighborsClassifier(n_neighbors=5)
KNN_.fit(X_train, y_train)
     
KNeighborsClassifier()

y_pred=KNN_.predict(X_test)
acc=classification_report(y_test, y_pred)
print(acc)
     

              precision    recall  f1-score   support

           0       0.83      0.83      0.83       654
           1       0.75      0.76      0.75       446

    accuracy                           0.80      1100
   macro avg       0.79      0.79      0.79      1100
weighted avg       0.80      0.80      0.80      1100



In [60]:
grid_params = { 'n_neighbors' : [5,7,9,11,13,15],
               'weights' : ['uniform','distance'],
               'metric' : ['minkowski','euclidean','manhattan']}
     

clf = GridSearchCV(KNN_,grid_params)
     

clf.fit(X_train,y_train)
     
clf.best_params_

{'metric': 'minkowski', 'n_neighbors': 13, 'weights': 'uniform'}

In [62]:

KNN_ = KNeighborsClassifier(metric = 'minkowski',n_neighbors=13,weights='uniform')
KNN_.fit(X_train, y_train)

y_pred=KNN_.predict(X_test)
acc=classification_report(y_test, y_pred)
print(acc)

              precision    recall  f1-score   support

           0       0.85      0.83      0.84       654
           1       0.76      0.79      0.77       446

    accuracy                           0.81      1100
   macro avg       0.81      0.81      0.81      1100
weighted avg       0.81      0.81      0.81      1100



In [61]:
clf.cv_results_

{'mean_fit_time': array([0.00448999, 0.00473161, 0.00447168, 0.00458388, 0.00501561,
        0.00403447, 0.0053997 , 0.00431232, 0.0044271 , 0.00440702,
        0.00465736, 0.00553565, 0.00515351, 0.00440669, 0.0043674 ,
        0.00461478, 0.00428233, 0.00422621, 0.00430198, 0.00412774,
        0.00436149, 0.00409245, 0.00421529, 0.00417819, 0.00440025,
        0.00428257, 0.00432897, 0.00524235, 0.00447659, 0.00428443,
        0.00438437, 0.00427675, 0.00456572, 0.00421448, 0.00500531,
        0.0044445 ]),
 'std_fit_time': array([1.10836959e-04, 5.40094894e-04, 9.92252870e-05, 6.50549258e-04,
        1.46902781e-03, 8.70362232e-05, 1.69989558e-03, 3.78809787e-04,
        1.44750562e-04, 1.55768081e-04, 3.22192294e-04, 1.21326465e-03,
        1.84838428e-03, 4.63264711e-04, 1.69112259e-04, 8.01547114e-04,
        1.48602198e-04, 1.85395673e-04, 2.04107103e-04, 3.42546453e-04,
        2.60444230e-04, 1.44627233e-04, 2.02997559e-04, 1.15991518e-04,
        1.35201204e-04, 1.28719945e-0

## **RBF SVC**


In [63]:
from sklearn.svm import SVC
     

model = Pipeline((('scaler',StandardScaler()), ('rbf_svm',SVC(random_state=0, gamma='auto'))))
model.fit(X_train,y_train)


y_pred=model.predict(X_test)
acc=classification_report(y_test, y_pred)
print(acc)

              precision    recall  f1-score   support

           0       0.85      0.86      0.86       654
           1       0.79      0.78      0.79       446

    accuracy                           0.83      1100
   macro avg       0.82      0.82      0.82      1100
weighted avg       0.83      0.83      0.83      1100



### **Grid Search**

In [64]:
clf = GridSearchCV(model,{'rbf_svm__C': [1,10,100,1000]})
     

clf.fit(X_train,y_train)
     
GridSearchCV(estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('rbf_svm',
                                        SVC(C=1000, gamma='auto',
                                            random_state=0))]),
             param_grid={'rbf_svm__C': [1, 10, 100, 1000]})

clf.best_params_

{'rbf_svm__C': 100}

In [65]:
model = Pipeline((('scaler',StandardScaler()), ('rbf_svm',SVC(random_state=0,C =1000, gamma='auto'))))
model.fit(X_train,y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('rbf_svm', SVC(C=1000, gamma='auto', random_state=0))])

In [66]:
y_pred=model.predict(X_test)
acc=classification_report(y_test, y_pred)
print(acc)

              precision    recall  f1-score   support

           0       0.86      0.86      0.86       654
           1       0.79      0.79      0.79       446

    accuracy                           0.83      1100
   macro avg       0.82      0.82      0.82      1100
weighted avg       0.83      0.83      0.83      1100



In [67]:
clf.cv_results_

{'mean_fit_time': array([0.4914319 , 0.56151195, 1.68384199, 8.54434395]),
 'std_fit_time': array([0.09713847, 0.0319285 , 0.32847977, 1.13723189]),
 'mean_score_time': array([0.07797704, 0.06059031, 0.09334745, 0.0908618 ]),
 'std_score_time': array([0.02240408, 0.00422612, 0.02888024, 0.02856632]),
 'param_rbf_svm__C': masked_array(data=[1, 10, 100, 1000],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'rbf_svm__C': 1},
  {'rbf_svm__C': 10},
  {'rbf_svm__C': 100},
  {'rbf_svm__C': 1000}],
 'split0_test_score': array([0.80681818, 0.80681818, 0.80681818, 0.81022727]),
 'split1_test_score': array([0.81136364, 0.81136364, 0.81136364, 0.81022727]),
 'split2_test_score': array([0.83636364, 0.83636364, 0.83863636, 0.83522727]),
 'split3_test_score': array([0.83409091, 0.83522727, 0.83522727, 0.83636364]),
 'split4_test_score': array([0.80113636, 0.80227273, 0.80227273, 0.80227273]),
 'mean_test_score': array([0.81795455, 0.8

# **Light GBM**



In [68]:
lgbm = LGBMClassifier()
model = LGBMClassifier()
model.fit(X_train,y_train)
     
y_pred = model.predict(X_test)
acc = classification_report(y_test,y_pred)
print(acc)

              precision    recall  f1-score   support

           0       0.87      0.84      0.85       654
           1       0.77      0.82      0.80       446

    accuracy                           0.83      1100
   macro avg       0.82      0.83      0.82      1100
weighted avg       0.83      0.83      0.83      1100



In [69]:
clf = GridSearchCV(model,dict(num_leaves=[5,10,100,1000],max_depth = [2,3,4,5,6]))
     
clf.fit(X_train,y_train)
     
clf.best_params_

{'max_depth': 4, 'num_leaves': 5}

In [70]:
model = LGBMClassifier(num_leaves = 100,max_depth = 4)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
acc = classification_report(y_test,y_pred)
print(acc)

              precision    recall  f1-score   support

           0       0.87      0.84      0.85       654
           1       0.78      0.82      0.80       446

    accuracy                           0.83      1100
   macro avg       0.82      0.83      0.83      1100
weighted avg       0.83      0.83      0.83      1100



In [71]:
clf.cv_results_

{'mean_fit_time': array([0.02750416, 0.02489486, 0.02455082, 0.02612462, 0.02755613,
        0.02957263, 0.03256087, 0.02965183, 0.03043551, 0.03256707,
        0.03577971, 0.03732023, 0.02743635, 0.03602986, 0.0434629 ,
        0.0442523 , 0.02844582, 0.03456101, 0.05278463, 0.05120049]),
 'std_fit_time': array([0.00260994, 0.00057849, 0.00043223, 0.00207419, 0.00144159,
        0.00205328, 0.00402949, 0.00117609, 0.00672678, 0.00024319,
        0.00227098, 0.00169806, 0.00175147, 0.00235384, 0.00240331,
        0.00313521, 0.00336709, 0.00115647, 0.00254447, 0.00184992]),
 'mean_score_time': array([0.00448813, 0.00399356, 0.00399537, 0.00408773, 0.0040576 ,
        0.00461125, 0.00466504, 0.00426755, 0.00434542, 0.00492964,
        0.00526471, 0.00589428, 0.00405126, 0.00611238, 0.00569205,
        0.00570645, 0.00410781, 0.00576591, 0.00699801, 0.00734591]),
 'std_score_time': array([7.76950593e-04, 1.13325786e-04, 7.07518371e-05, 3.00460667e-04,
        1.74891129e-04, 3.85894299e-

In [74]:
X_PIL_train, X_PIL_test, y_PIL_train, y_PIL_test=train_test_split(PIL_img_data ,class_name,stratify=class_name,train_size=0.8,random_state=10)

In [75]:
XX=np.asarray(X_PIL_train)
YY=np.asarray(y_PIL_train)
xx=np.asarray(X_PIL_test)
yy=np.asarray(y_PIL_test)
xx.shape
     

(1100, 96, 96, 3)

In [76]:
x1=np.swapaxes(XX,1,3)
x1=np.swapaxes(x1,2,3)
x1.shape

(4400, 3, 96, 96)

In [77]:
x2=np.swapaxes(xx,1,3)
x2=np.swapaxes(x2,2,3)
x2.shape

(1100, 3, 96, 96)

In [78]:
class Dataset(Dataset):
    def __init__(self, X,y, transform=None):
        super().__init__()
        self.X = torch.from_numpy(X).float()
        self.y = y
        self.transform = transform

    def __len__(self):
        return len(self.y)

    def __getitem__(self, index):
        image= self.X[index]
        label =self.y[index]
        if self.transform is not None:
            image = self.transform(image)
        return image, label

In [79]:
batch_size = 128
valid_size = 0.1

In [90]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_epochs = 3
num_classes = 2
learning_rate = 0.01

In [81]:
def create_dataset_PIL(img_folder,labels):
    
    img_data_array=[]
    class_name=[]
    for file in range(len(labels)):
       
            image_path= os.path.join(img_folder,  labels['id'][file]+'.tif')
            image= np.array(Image.open(image_path))
            image = image.astype('float32')
            image /= 255  
            img_data_array.append(image)
            class_name.append(labels['label'][file])
    return img_data_array , class_name
PIL_img_data, class_name=create_dataset_PIL(img_folder,labels)

In [85]:
trans_train = transforms.Compose([transforms.ToPILImage(), transforms.Pad(64, padding_mode='reflect'),transforms.RandomHorizontalFlip(), 
                                  transforms.RandomVerticalFlip(), transforms.RandomRotation(20),             
                                  transforms.ToTensor(), transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])])    

trans_valid = transforms.Compose([transforms.ToPILImage(), transforms.Pad(64, padding_mode='reflect'),
                                  transforms.ToTensor(), transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])])

In [86]:
dataset_train = Dataset(X=x1, y=y_PIL_train, transform=trans_train)
dataset_valid = Dataset(X=x2, y=y_PIL_test, transform=trans_valid)

train_loader = DataLoader(dataset = dataset_train, batch_size=batch_size, shuffle=True, num_workers=0)
valid_loader = DataLoader(dataset = dataset_valid, batch_size=batch_size//2, shuffle=False, num_workers=0)

In [91]:

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential( nn.Conv2d(3, 32, kernel_size=3, padding=2),
                                    nn.BatchNorm2d(32), nn.ReLU(), nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer2 = nn.Sequential(nn.Conv2d( 32, 64, kernel_size=3, padding=2),
                    nn.BatchNorm2d(64),nn.ReLU(),nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer3 = nn.Sequential(nn.Conv2d(64, 128, kernel_size=3, padding=2),
                                nn.BatchNorm2d(128), nn.ReLU(), nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer4 = nn.Sequential( nn.Conv2d(128, 256, kernel_size=3, padding=2),
                                nn.BatchNorm2d(256), nn.ReLU(), nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer5 = nn.Sequential( nn.Conv2d(256, 512, kernel_size=3, padding=2),
            nn.BatchNorm2d(512), nn.ReLU(), nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.avg = nn.AvgPool2d(8)
        self.fc = nn.Linear(512 * 1 * 1, 2)
        
    def forward(self,x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.avg(x)
        x = x.view(-1, 512 * 1 * 1)
        x = self.fc(x)
        return x
     

In [88]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = CNN().to(device)
print(model)

CNN(
  (layer1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer4): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): BatchNor

In [92]:

criterion = nn.CrossEntropyLoss()


optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)


train_losses = []
val_losses = []
train_auc = []
val_auc = []
train_auc_epoch = []
val_auc_epoch = []
best_acc = 0.0
min_loss = np.Inf

since = time.time()

for e in range(num_epochs):
    
    train_loss = 0.0
    val_loss = 0.0
    
    model.train()
    for i, (images, labels) in enumerate(tqdm(train_loader, total=int(len(train_loader)))):
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        y_actual = labels.data.cpu().numpy()
        y_pred = outputs[:,-1].detach().cpu().numpy()
        train_auc.append(roc_auc_score(y_actual, y_pred))
    

    model.eval()
    for i, (images, labels) in enumerate(tqdm(valid_loader, total=int(len(valid_loader)))):
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        

        val_loss += loss.item()
        y_actual = labels.data.cpu().numpy()
        y_pred = outputs[:,-1].detach().cpu().numpy()
        val_auc.append(roc_auc_score(y_actual, y_pred))
    
    train_loss = train_loss/len(train_loader)
    val_loss = val_loss/len(valid_loader)
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    training_auc = np.mean(train_auc)
    validation_auc = np.mean(val_auc)
    train_auc_epoch.append(training_auc)
    val_auc_epoch.append(validation_auc)
    

    if best_acc < validation_auc:
        best_acc = validation_auc
        
    if min_loss >= val_loss:
        torch.save(model.state_dict(), 'best_model.pt')
        min_loss = val_loss
    
    print('EPOCH {}/{} Train loss: {:.6f},Validation loss: {:.6f}, Train AUC: {:.4f}  Validation AUC: {:.4f}\n  '.format(e+1, num_epochs,train_loss,val_loss, training_auc,validation_auc))
    print('-' * 10)
time_elapsed = time.time() - since
print('Training completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
print('Best validation accuracy: {:4f}'.format(best_acc))

  0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

EPOCH 1/3 Train loss: 0.536515,Validation loss: 0.833903, Train AUC: 0.8501  Validation AUC: 0.7313
  
----------


  0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

EPOCH 2/3 Train loss: 0.443584,Validation loss: 0.434429, Train AUC: 0.8622  Validation AUC: 0.8042
  
----------


  0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

EPOCH 3/3 Train loss: 0.425921,Validation loss: 0.629089, Train AUC: 0.8700  Validation AUC: 0.8315
  
----------
Training completed in 67m 16s
Best validation accuracy: 0.831522


In [ ]:
plt.figure(figsize=(20,5))
plt.plot(train_losses, '-o', label="train")
plt.plot(val_losses, '-o', label="val")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Loss change over epoch")
plt.legend()

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(train_auc_epoch, '-o', label="train")
plt.plot(val_auc_epoch, '-o', label="val")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Accuracy over epoch")
plt.legend()
     